In [ ]:
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)


In [ ]:
def previous_applications(df,num_rows = None, nan_as_category = True):
    df_prev = df.copy()
    a = ['Family', 'Spouse, partner', 'Children', 'Other_B', 'Other_A', 'Group of people']
    df_prev["NAME_TYPE_SUITE"] = df_prev["NAME_TYPE_SUITE"].replace(a, 'Accompanied')
    
    b = ['Auto Accessories', 'Jewelry', 'Homewares', 'Medical Supplies', 'Vehicles', 'Sport and Leisure', 
         'Gardening', 'Other', 'Office Appliances', 'Tourism', 'Medicine', 'Direct Sales', 'Fitness', 'Additional Service', 
         'Education', 'Weapon', 'Insurance', 'House Construction', 'Animals'] 
    df_prev["NAME_GOODS_CATEGORY"] = df_prev["NAME_GOODS_CATEGORY"].replace(b, 'others')
    
    c = ['AP+ (Cash loan)', 'Channel of corporate sales', 'Car dealer']
    df_prev["CHANNEL_TYPE"] = df_prev["CHANNEL_TYPE"].replace(c, 'Other_Channel')
    
    d = ['Auto technology', 'Jewelry', 'MLM partners', 'Tourism']
    df_prev["NAME_SELLER_INDUSTRY"] = df_prev["NAME_SELLER_INDUSTRY"].replace(d, 'Others')
    
    e = ['Refusal to name the goal', 'Money for a third person', 'Buying a garage','Gasification / water supply',
     'Hobby','Business development','Buying a holiday home / land','Furniture','Car repairs',
     'Buying a home','Wedding / gift / holiday']
    df_prev["NAME_CASH_LOAN_PURPOSE"] = df_prev["NAME_CASH_LOAN_PURPOSE"].replace(e, 'Other_Loan')
    
    
    df_prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    df_prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    df_prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    df_prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    df_prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    
  
    df_prev['NEW_APP_CREDIT_RATE'] = df_prev['AMT_APPLICATION'] / df_prev['AMT_CREDIT']
    
    df_prev["NEW_APP_CREDIT_RATE_RATIO"] = df_prev["NEW_APP_CREDIT_RATE"].apply(lambda x: 1 if(x<=1) else 0)
    df_prev['NEW_AMT_PAYMENT_RATE'] = df_prev['AMT_CREDIT'] / df_prev['AMT_ANNUITY']

    df_prev['NEW_APP_GOODS_RATE'] = df_prev['AMT_APPLICATION'] / df_prev['AMT_GOODS_PRICE']

    df_prev['NEW_CREDIT_GOODS_RATE'] = df_prev['AMT_CREDIT'] / df_prev['AMT_GOODS_PRICE']

    df_prev['NEW_RETURN_DAY'] =  df_prev['DAYS_DECISION'] + df_prev['CNT_PAYMENT'] * 30

    df_prev['NEW_DAYS_TERMINATION_DIFF'] = df_prev['DAYS_TERMINATION'] - df_prev['NEW_RETURN_DAY']

    df_prev['NEW_DAYS_DUE_DIFF'] = df_prev['DAYS_LAST_DUE_1ST_VERSION'] - df_prev['DAYS_FIRST_DUE'] 
    
    df_prev["NEW_CNT_PAYMENT"] = pd.cut(x=df_prev['CNT_PAYMENT'], bins=[0, 12, 60,120], labels=["Kısa", "Orta", "Uzun"])
    
    df_prev["NEW_END_DIFF"] = df_prev["DAYS_TERMINATION"] - df_prev["DAYS_LAST_DUE"]

    weekend = ["SATURDAY","SUNDAY"]                              
    df_prev["WEEKDAY_APPR_PROCESS_START"] = df_prev["WEEKDAY_APPR_PROCESS_START"].apply(lambda x : "WEEKEND" if (x in weekend) else "WEEKDAY")
    
      
    df_prev['NFLAG_LAST_APPL_IN_DAY'] = df_prev['NFLAG_LAST_APPL_IN_DAY'].astype("O")
    df_prev['FLAG_LAST_APPL_PER_CONTRACT'] = df_prev['FLAG_LAST_APPL_PER_CONTRACT'].astype("O")
    df_prev["NEW_CNT_PAYMENT"] = df_prev['NEW_CNT_PAYMENT'].astype("O")
    df_prev['NEW_APP_CREDIT_RATE_RATIO'] = df_prev['NEW_APP_CREDIT_RATE_RATIO'].astype('O')
    newCoding = {"0": "Yes", "1": "No"}
    df_prev['NEW_APP_CREDIT_RATE_RATIO'] = df_prev['NEW_APP_CREDIT_RATE_RATIO'].replace(newCoding)
    
    df_prev, cat_cols = one_hot_encoder(df_prev, nan_as_category= True)
    
    # Aggregation for numeric features
    num_aggregations = {
        'SK_ID_PREV' : 'count',
        'AMT_ANNUITY': ['min', 'max', 'median', 'mean'], 
        'AMT_APPLICATION': ['min', 'max', 'mean', 'median'],
        'AMT_CREDIT': ['min', 'max', 'mean', 'median'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean', 'median'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean', 'median'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean','median'], 
        'DAYS_DECISION': ['min', 'max', 'mean','median'],
        'NEW_APP_CREDIT_RATE': ['min', 'max', 'mean', 'var'],
        'NEW_AMT_PAYMENT_RATE': ['min', 'max', 'mean'],
        'NEW_APP_GOODS_RATE':['min', 'max', 'mean'],
        'NEW_CREDIT_GOODS_RATE': ['min', 'max', 'mean'],
        'NEW_RETURN_DAY': ['min', 'max', 'mean', 'var'],
        'NEW_DAYS_TERMINATION_DIFF': ['min', 'max', 'mean'],
        'NEW_END_DIFF': ['min', 'max', 'mean'],
        'NEW_APP_CREDIT_RATE_RATIO': ['min', 'max', 'mean'],
        'NEW_DAYS_DUE_DIFF': ['min', 'max', 'mean']       
    }
     
    # Aggregation for categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = df_prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    
    # Approved Applications - Aggregation for numeric features
    approved = df_prev[df_prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    
    #Refused Applications - Aggregation for numeric features
    refused = df_prev[df_prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    
    del refused, refused_agg, approved, approved_agg, df_prev
    gc.collect()
    return prev_agg

In [ ]:
df = pd.read_csv('/content/previous_application.csv')

In [ ]:
df2 = previous_applications(df)
df2

,PREV_SK_ID_PREV_COUNT,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEDIAN,PREV_AMT_ANNUITY_MEAN,PREV_AMT_APPLICATION_MIN,PREV_AMT_APPLICATION_MAX,PREV_AMT_APPLICATION_MEAN,PREV_AMT_APPLICATION_MEDIAN,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,PREV_AMT_CREDIT_MEAN,PREV_AMT_CREDIT_MEDIAN,PREV_AMT_DOWN_PAYMENT_MIN,PREV_AMT_DOWN_PAYMENT_MAX,PREV_AMT_DOWN_PAYMENT_MEAN,PREV_AMT_DOWN_PAYMENT_MEDIAN,PREV_AMT_GOODS_PRICE_MIN,PREV_AMT_GOODS_PRICE_MAX,PREV_AMT_GOODS_PRICE_MEAN,PREV_AMT_GOODS_PRICE_MEDIAN,PREV_HOUR_APPR_PROCESS_START_MIN,PREV_HOUR_APPR_PROCESS_START_MAX,PREV_HOUR_APPR_PROCESS_START_MEAN,PREV_RATE_DOWN_PAYMENT_MIN,PREV_RATE_DOWN_PAYMENT_MAX,PREV_RATE_DOWN_PAYMENT_MEAN,PREV_RATE_DOWN_PAYMENT_MEDIAN,PREV_DAYS_DECISION_MIN,PREV_DAYS_DECISION_MAX,PREV_DAYS_DECISION_MEAN,PREV_DAYS_DECISION_MEDIAN,PREV_NEW_APP_CREDIT_RATE_MIN,PREV_NEW_APP_CREDIT_RATE_MAX,PREV_NEW_APP_CREDIT_RATE_MEAN,PREV_NEW_APP_CREDIT_RATE_VAR,PREV_NEW_AMT_PAYMENT_RATE_MIN,PREV_NEW_AMT_PAYMENT_RATE_MAX,PREV_NEW_AMT_PAYMENT_RATE_MEAN,PREV_NEW_APP_GOODS_RATE_MIN,PREV_NEW_APP_GOODS_RATE_MAX,PREV_NEW_APP_GOODS_RATE_MEAN,PREV_NEW_CREDIT_GOODS_RATE_MIN,PREV_NEW_CREDIT_GOODS_RATE_MAX,PREV_NEW_CREDIT_GOODS_RATE_MEAN,PREV_NEW_RETURN_DAY_MIN,PREV_NEW_RETURN_DAY_MAX,PREV_NEW_RETURN_DAY_MEAN,PREV_NEW_RETURN_DAY_VAR,PREV_NEW_DAYS_TERMINATION_DIFF_MIN,PREV_NEW_DAYS_TERMINATION_DIFF_MAX,PREV_NEW_DAYS_TERMINATION_DIFF_MEAN,PREV_NEW_END_DIFF_MIN,PREV_NEW_END_DIFF_MAX,PREV_NEW_END_DIFF_MEAN,PREV_NEW_APP_CREDIT_RATE_RATIO_MIN,PREV_NEW_APP_CREDIT_RATE_RATIO_MAX,PREV_NEW_APP_CREDIT_RATE_RATIO_MEAN,PREV_NEW_DAYS_DUE_DIFF_MIN,PREV_NEW_DAYS_DUE_DIFF_MAX,PREV_NEW_DAYS_DUE_DIFF_MEAN,PREV_NAME_CONTRACT_TYPE_Cash loans_MEAN,PREV_NAME_CONTRACT_TYPE_Consumer loans_MEAN,PREV_NAME_CONTRACT_TYPE_Revolving loans_MEAN,PREV_NAME_CONTRACT_TYPE_XNA_MEAN,PREV_NAME_CONTRACT_TYPE_nan_MEAN,PREV_WEEKDAY_APPR_PROCESS_START_WEEKDAY_MEAN,PREV_WEEKDAY_APPR_PROCESS_START_WEEKEND_MEAN,PREV_WEEKDAY_APPR_PROCESS_START_nan_MEAN,PREV_FLAG_LAST_APPL_PER_CONTRACT_N_MEAN,PREV_FLAG_LAST_APPL_PER_CONTRACT_Y_MEAN,PREV_FLAG_LAST_APPL_PER_CONTRACT_nan_MEAN,PREV_NFLAG_LAST_APPL_IN_DAY_0.0_MEAN,PREV_NFLAG_LAST_APPL_IN_DAY_1.0_MEAN,PREV_NFLAG_LAST_APPL_IN_DAY_nan_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Building a house or an annex_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Buying a new car_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Buying a used car_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Education_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Everyday expenses_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Journey_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Medicine_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Other_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Other_Loan_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Payments on other loans_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Purchase of electronic equipment_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Repairs_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_Urgent needs_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_XAP_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_XNA_MEAN,PREV_NAME_CASH_LOAN_PURPOSE_nan_MEAN,PREV_NAME_CONTRACT_STATUS_Approved_MEAN,PREV_NAME_CONTRACT_STATUS_Canceled_MEAN,PREV_NAME_CONTRACT_STATUS_Refused_MEAN,PREV_NAME_CONTRACT_STATUS_Unused offer_MEAN,PREV_NAME_CONTRACT_STATUS_nan_MEAN,PREV_NAME_PAYMENT_TYPE_Cash through the bank_MEAN,PREV_NAME_PAYMENT_TYPE_Cashless from the account of the employer_MEAN,PREV_NAME_PAYMENT_TYPE_Non-cash from your account_MEAN,PREV_NAME_PAYMENT_TYPE_XNA_MEAN,PREV_NAME_PAYMENT_TYPE_nan_MEAN,PREV_CODE_REJECT_REASON_CLIENT_MEAN,PREV_CODE_REJECT_REASON_HC_MEAN,PREV_CODE_REJECT_REASON_LIMIT_MEAN,PREV_CODE_REJECT_REASON_SCO_MEAN,PREV_CODE_REJECT_REASON_SCOFR_MEAN,PREV_CODE_REJECT_REASON_SYSTEM_MEAN,PREV_CODE_REJECT_REASON_VERIF_MEAN,PREV_CODE_REJECT_REASON_XAP_MEAN,PREV_CODE_REJECT_REASON_XNA_MEAN,PREV_CODE_REJECT_REASON_nan_MEAN,PREV_NAME_TYPE_SUITE_Accompanied_MEAN,PREV_NAME_TYPE_SUITE_Unaccompanied_MEAN,PREV_NAME_TYPE_SUITE_nan_MEAN,PREV_NAME_CLIENT_TYPE_New_MEAN,PREV_NAME_CLIENT_TYPE_Refreshed_MEAN,PREV_NAME_CLIENT_TYPE_Repeater_MEAN,PREV_NAME_CLIENT_TYPE_XNA_MEAN,PREV_NAME_CL

In [ ]:
df2.head(100).to_csv('python_output.csv')